In [ ]:
import sys, os, glob, yaml

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import pprint
import seaborn as sns
import trackml.dataset

In [ ]:
import torch
from torch_geometric.data import Data
import itertools

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# local imports
from src import Compose_Event, Draw_Compose_Event
from src import SttCSVDataReader, Draw_CSVReader_Event, SttTorchDataReader, Draw_TorchReader_Event

### _Dataset_

In [ ]:
# mu+mu- data (current)
input_dir = './train_quick'

In [ ]:
# Find All Input Data Files (hits.csv, cells.csv, particles.csv, truth.csv)
all_files = os.listdir(input_dir)

# Extract File Prefixes (use e.g. xxx-hits.csv)
suffix = '-hits.csv'
file_prefixes = sorted(os.path.join(input_dir, f.replace(suffix, ''))
                       for f in all_files if f.endswith(suffix))

print("Total Files: ", len(file_prefixes))

In [ ]:
# load an event
event_prefix = file_prefixes[4]
hits, tubes, particles, truth = trackml.dataset.load_event(event_prefix)

# memory usage
mem_bytes = (hits.memory_usage(index=True).sum() 
             + tubes.memory_usage(index=True).sum() 
             + particles.memory_usage(index=True).sum() 
             + truth.memory_usage(index=True).sum())

print('{} memory usage {:.2f} MB'.format(os.path.basename(event_prefix), mem_bytes / 2**20))

In [ ]:
# hits.head()
# tubes.head()
# particles.head()
# truth.head()

### SttCSVReader

In [ ]:
# reader = SttCSVReader(path=input_dir,selection=True,noise=False,skewed=False)

In [ ]:
# event_id = 4
# read = reader(event_id)

In [ ]:
# read.event.pdgcode.unique()

In [ ]:
# Draw_Reader_Event(data=read,figsize=(9,9),save_fig=False);

### _Compose Event_
- _`Compose_Event()` is same as `select_hits()` in `processing/utils/event_utils.py`_

In [ ]:
event = Compose_Event(event_prefix, selection=False, noise=False, skewed=False)
Draw_Compose_Event(event,figsize=(9,9),save_fig=False);

- _test if `selction` flag works_

event = Compose_Event(event_prefix, selection=True, noise=False, skewed=False)
Draw_Compose_Event(event,figsize=(9,9),save_fig=False);

- add decay vertex parameters `d0 = sqrt(vx**2+vy**2), z0=vz`

In [ ]:
from LightningModules.Processing.utils.event_utils import select_hits

In [ ]:
kwarg={"selection": False}

In [ ]:
event = select_hits(event_file=event_prefix, noise=False, skewed=False, **kwarg)

In [ ]:
event.vx

In [ ]:
e = event.iloc[0]

In [ ]:
np.sqrt(e.vx**2 + e.vy**2)

- _further analysis_

In [ ]:
# read.event_file

In [ ]:
# read.hits.head()
# read.tubes.head()
# read.particles.head()
# read.event.head()

In [ ]:
# read.event.layer.unique()

In [ ]:
particles.head()

- _Drop Duplicates in "Particles"_

In [ ]:
# particles = event.particles.copy() # copy() it otherwise event.particles is changed
# particles['nhits'] = particles.groupby(['particle_id'])['nhits'].transform('count')
# particles.drop_duplicates(inplace=True, ignore_index=True)
# particles.head(10)

- _Filter some particles in "Particles"_

In [ ]:
# there are particle we don't need e.g. electron (11), (1000010020), etc
particles.pdgcode.unique()

In [ ]:
# see all particle in the particle dataframe
particles.head()

In [ ]:
# just keep protons, pions and maybe muon, don't forget resetting index and dropping old one.
particles[particles['pdgcode'].isin([-2212, 2212, -211, 211])].reset_index(drop=True)

- _renaming layers_

In [ ]:
# hits.head()

In [ ]:
# vlids = hits.layer_id.unique()
# n_det_layers == len(vlids)
# vlid_groups = hits.groupby(['layer_id'])
# hits = pd.concat([vlid_groups.get_group(vlids[i]).assign(layer_id=i) for i in range(n_det_layers)])  # need reset index

In [ ]:
# hits.head()

### _Test Processed Data_

In [ ]:
from src.drawing import detector_layout, draw_proc_event

In [ ]:
inputdir="run_quick/feature_store_fwp"
num_files = sorted(glob.glob(os.path.join(inputdir, "*")))

In [ ]:
print("Number of loaded files: ", len(num_files))

In [ ]:
data = torch.load(num_files[0], map_location=device)

In [ ]:
data

In [ ]:
event_id = int(os.path.basename(data.event_file)[-4:])

In [ ]:
r  = data.x[:, 0]
phi= data.x[:, 1]
ir = data.x[:, 2]

In [ ]:
r

In [ ]:
vx = data.vertex[:, 0]
vy = data.vertex[:, 1] 
vz = data.vertex[:, 2]

In [ ]:
vx

In [ ]:
vy

In [ ]:
np.sqrt(vx[0]**2+vy[0]**2)

In [ ]:
d0 = torch.sqrt(vx**2+vy**2)

In [ ]:
d0[0]

In [ ]:
# mpl.rcParams.update(mpl.rcParamsDefault)